In [70]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score

In [80]:
training_data = pd.read_csv('train.csv')


In [81]:
#CryoSleep and VIP = boolean
#destination and home planet = groups
#cabin needs to be split into 3 separate columns
print(training_data['Destination'].unique())
'''training_data['CryoSleep'] = training_data['CryoSleep'].map({'True': 1, 'False': 0})
training_data['VIP'] = training_data['VIP'].map({'True': 1, 'False': 0})
'''
training_data['HomePlanet'] = training_data['HomePlanet'].map({'Europa': 0, 'Earth': 1, 'Mars':2})
training_data['Destination'] = training_data['Destination'].map({'TRAPPIST-1e': 0, 'PSO J318.5-22': 1, '55 Cancri e':2})
training_data[['Deck', 'Num', 'Side']] = training_data['Cabin'].str.split('/', expand=True)
training_data = training_data.drop(columns=['Cabin'])
deck_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'T': 7}
training_data['Deck'] = training_data['Deck'].map(deck_map)
training_data['Side'] = training_data['Side'].map({'P': 0, 'S': 1})
training_data['CryoSleep'] = training_data['CryoSleep'].fillna(False).astype(int)
training_data['VIP'] = training_data['VIP'].fillna(False).astype(int)
training_data['Transported'] = training_data['Transported'].fillna(False).astype(int)


training_data


['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]


,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side
0,0001_01,0.0,0,0.0,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,1.0,0,0.0
1,0002_01,1.0,0,0.0,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,5.0,0,1.0
2,0003_01,0.0,0,0.0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,0.0,0,1.0
3,0003_02,0.0,0,0.0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,0.0,0,1.0
4,0004_01,1.0,0,0.0,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,5.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.0,0,2.0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0,0.0,98,0.0
8689,9278_01,1.0,1,1.0,18.0,0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0,6.0,1499,1.0
8690,9279_01,1.0,0,0.0,26.0,0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1,6.0,1500,1.0
8691,9280_01,0.0,0,2.0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0,4.0,608,1.0


In [73]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

In [82]:
training_data['Deck'] = encoder.fit_transform(training_data[['Deck']])
training_data['Side'] = encoder.fit_transform(training_data[['Side']])
training_data

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side
0,0001_01,0.0,0,0.0,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,1.0,0,0.0
1,0002_01,1.0,0,0.0,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,5.0,0,1.0
2,0003_01,0.0,0,0.0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,0.0,0,1.0
3,0003_02,0.0,0,0.0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,0.0,0,1.0
4,0004_01,1.0,0,0.0,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,5.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.0,0,2.0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0,0.0,98,0.0
8689,9278_01,1.0,1,1.0,18.0,0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0,6.0,1499,1.0
8690,9279_01,1.0,0,0.0,26.0,0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1,6.0,1500,1.0
8691,9280_01,0.0,0,2.0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0,4.0,608,1.0


In [83]:
training_data = training_data.dropna()
target = training_data['Transported']
training_data = training_data.drop(columns=['Transported', 'PassengerId','Name'])

training_data

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
0,0.0,0,0.0,39.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0
1,1.0,0,0.0,24.0,0,109.0,9.0,25.0,549.0,44.0,5.0,0,1.0
2,0.0,0,0.0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0.0,0,1.0
3,0.0,0,0.0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0.0,0,1.0
4,1.0,0,0.0,16.0,0,303.0,70.0,151.0,565.0,2.0,5.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,0,2.0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0.0,98,0.0
8689,1.0,1,1.0,18.0,0,0.0,0.0,0.0,0.0,0.0,6.0,1499,1.0
8690,1.0,0,0.0,26.0,0,0.0,0.0,1872.0,1.0,0.0,6.0,1500,1.0
8691,0.0,0,2.0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,4.0,608,1.0


In [84]:

features  = training_data.columns.tolist()

preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), features)

        ]
    )

processed_data = preprocessor.fit_transform(training_data)
X_train, X_test, y_train, y_test = train_test_split(processed_data, target, test_size=0.1, random_state=42, shuffle=False)
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)


GradientBoostingRegressor(random_state=42)

In [87]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
y_pred = model.predict(X_test)
# Convert predictions to 0/1 if needed

y_pred = (y_pred >= 0.5).astype(int)
y_pred = np.array(y_pred)
np_y_test = np.array(y_test)
print(np.unique(np_y_test), np_y_test.dtype)
print(np.unique(y_pred), y_pred.dtype)


acc = accuracy_score(np_y_test, y_pred)
prec = precision_score(np_y_test, y_pred)
rec = recall_score(np_y_test, y_pred)
f1 = f1_score(np_y_test, y_pred)

print(f"\nAccuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:\n", confusion_matrix(np_y_test, y_pred))


[0 1] int64
[0 1] int64

Accuracy: 0.7856
Precision: 0.7209
Recall: 0.8719
F1 Score: 0.7893
Confusion Matrix:
 [[267 108]
 [ 41 279]]
